In [1]:
import os
import tensorflow as tf
import dask
import dask.array as da
import numpy as np
from PIL import Image
from skimage.transform import resize
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_x')
train_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/train_y')
test_x = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_x')
test_y = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_y')
test_fov = da.from_npy_stack('/home/skyolia/JupyterProjects/data/DRIVE/test_fov')
train_x, train_y, test_x, test_y, test_fov

(dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512, 1), dtype=float32, chunksize=(20, 512, 512, 1)>,
 dask.array<from-npy-stack, shape=(20, 512, 512), dtype=float32, chunksize=(20, 512, 512)>)

In [3]:
def get_inside_fov(test_fov):
    flat = test_fov.ravel()
    flat = da.where(flat>0.5, 1, 0)
    index_inside_fov = da.where(flat)[0]
    return index_inside_fov
inside_fov = get_inside_fov(test_fov).compute()
inside_fov

array([  14069,   14070,   14071, ..., 5235971, 5235972, 5235973])

In [4]:
class AUCTensorBoard(tf.keras.callbacks.TensorBoard):
    '''
    Add learning rate evolution to Tensorboard
    '''
    def __init__(self, log_dir, training_data, validation_data):
        super().__init__(log_dir=log_dir)
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs=None):
        bin_test_y = np.where(self.y_val.compute()>0.5, 1, 0)
        y_pred_val = self.model.predict(self.x_val, batch_size=1)
        roc_val = roc_auc_score(bin_test_y.ravel()[inside_fov], y_pred_val.ravel()[inside_fov])
        print('aucroc = ', roc_val)
        logs['auc'] = roc_val
        super().on_epoch_end(epoch, logs)
        
def build_block(input_layer, filters, norm=True, k=(3, 3)):
    layer = tf.keras.layers.Conv2D(filters, kernel_size=k, padding='same', use_bias=not norm, kernel_initializer='glorot_normal')(input_layer)
    if norm:
        layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Activation('elu')(layer)
    return layer

def build_unet(input_shape, n_filters=16, dropout=0.2):
    image_input = tf.keras.Input(shape=input_shape, name='input_layer')
    
    conv_1 = build_block(image_input, n_filters)
    conv_2 = build_block(conv_1, n_filters)
    pool_1 = tf.keras.layers.AveragePooling2D(padding='same')(conv_2)
    drop_1 = tf.keras.layers.SpatialDropout2D(dropout)(pool_1)
    
    conv_3 = build_block(drop_1, n_filters * 2)
    conv_4 = build_block(conv_3, n_filters * 2)
    pool_2 = tf.keras.layers.AveragePooling2D(padding='same')(conv_4)
    drop_2 = tf.keras.layers.SpatialDropout2D(dropout)(pool_2)
    
    conv_5 = build_block(drop_2, n_filters * 4)
    conv_6 = build_block(conv_5, n_filters * 4)
    pool_3 = tf.keras.layers.AveragePooling2D(padding='same')(conv_6)
    drop_3 = tf.keras.layers.SpatialDropout2D(dropout)(pool_3)
    
    conv_7 = build_block(drop_3, n_filters * 8)
    conv_8 = build_block(conv_7, n_filters * 8)
    pool_4 = tf.keras.layers.AveragePooling2D(padding='same')(conv_8)
    drop_4 = tf.keras.layers.SpatialDropout2D(dropout)(pool_4)
    
    conv_9 = build_block(drop_4, n_filters * 16)
    conv_10 = build_block(conv_9, n_filters * 16)
    
    upsp_1 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv_10) #(-1, 8, 8, 256)
    upsp_1 = tf.keras.layers.concatenate([upsp_1, conv_8])#(-1, 8, 8, 384)
    conv_11 = build_block(upsp_1, n_filters * 8)#(-1, 8, 8, 128)
    conv_12 = build_block(conv_11, n_filters * 8)#(-1, 8, 8, 128)
    drop_5 = tf.keras.layers.SpatialDropout2D(dropout)(conv_12)#(-1, 8, 8, 128)
    
    upsp_2 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_5) #(-1, 16, 16, 128)
    upsp_2 = tf.keras.layers.concatenate([upsp_2, conv_6]) #(-1, 16, 16, 192)
    conv_13 = build_block(upsp_2, n_filters * 4) #(-1, 16, 16, 64)
    conv_14 = build_block(conv_13, n_filters * 4) #(-1, 16, 16, 64)
    drop_6 = tf.keras.layers.SpatialDropout2D(dropout)(conv_14) #(-1, 16, 16, 64)
    
    upsp_3 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_6) #(-1, 32, 32, 64)
    upsp_3 = tf.keras.layers.concatenate([upsp_3, conv_4]) #(-1, 32, 32, 96)
    conv_15 = build_block(upsp_3, n_filters * 2) #(-1, 32, 32, 32)
    conv_16 = build_block(conv_15, n_filters * 2) #(-1, 32, 32, 32)
    drop_7 = tf.keras.layers.SpatialDropout2D(dropout)(conv_16) #(-1, 32, 32, 32)
    
    upsp_4 = tf.keras.layers.UpSampling2D(size=(2, 2))(drop_7) #(-1, 64, 64, 32)
    upsp_4 = tf.keras.layers.concatenate([upsp_4, conv_2])#(-1, 64, 64, 48)
    conv_17 = build_block(upsp_4, n_filters)#(-1, 64, 64, 16)
    conv_18 = build_block(conv_17, n_filters)#(-1, 64, 64, 16)
    drop_8 = tf.keras.layers.SpatialDropout2D(dropout)(conv_18)#(-1, 64, 64, 16)
    
    output = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer='glorot_normal', activation='sigmoid')(drop_8)
    model = tf.keras.Model(inputs=image_input, outputs=output)
    return model

def random_crop(img, msk, random_crop_size):
    # Note: image_data_format is 'channel_last'
    #assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :], msk[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_x_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        batch_y_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, batch_x.shape[3]))
        for i in range(batch_x.shape[0]):
            batch_x_crops[i], batch_y_crops[i] = random_crop(batch_x[i], batch_y[i], (crop_length, crop_length))
        yield (batch_x_crops, batch_y_crops)
        
model = build_unet(input_shape=(None, None, 1))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 1 144         input_layer[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 1 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 1 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [5]:
model.load_weights("day_1.weights.best.hdf5")
opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
scores = model.evaluate(test_x, test_y, batch_size=1)
scores

20/20 [==============================] - 3s 129ms/step


[0.09120763093233109, 0.921558952331543]

In [ ]:
epochs, batch_size, lr, filepath = 1000000, 4, 0.001, "day_2.weights.best.hdf5"
steps_per_epoch = int(np.ceil(train_y.shape[0]/batch_size))

data_gen_args = dict(horizontal_flip=True, vertical_flip=True) #width_shift_range=0.1, height_shift_range=0.1, 
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(train_x, augment=True, seed=seed)
mask_datagen.fit(train_y, augment=True, seed=seed)

image_generator = image_datagen.flow(x=train_x, batch_size=batch_size, seed=seed)
mask_generator = mask_datagen.flow(x=train_y, batch_size=batch_size, seed=seed)
train_generator = crop_generator(zip(image_generator, mask_generator), 64)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator().flow(x=test_x, y=test_y, batch_size=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#tb = tf.keras.callbacks.TensorBoard(log_dir=os.getcwd())
auc_roc = AUCTensorBoard(log_dir=os.getcwd(), training_data=(train_x, train_y),validation_data=(test_x, test_y))

opt = tf.keras.optimizers.Adam() # 
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.metrics_names)

model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=test_generator,
                    validation_steps=int(np.ceil(test_y.shape[0]/batch_size)),
                    use_multiprocessing=False,
                    workers=12,
                    shuffle=True,
                    initial_epoch=1257,
                    callbacks=[checkpoint, auc_roc])

['loss', 'acc']
Epoch 1258/1000000
3/5 [=================>............] - ETA: 3s - loss: 0.5321 - acc: 0.8957 
Epoch 01258: val_acc improved from -inf to 0.92273, saving model to day_2.weights.best.hdf5
aucroc =  0.9703499638835366
5/5 [==============================] - 11s 2s/step - loss: 0.4876 - acc: 0.9066 - val_loss: 0.0937 - val_acc: 0.9227
Epoch 1259/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5237 - acc: 0.8985
Epoch 01259: val_acc improved from 0.92273 to 0.92284, saving model to day_2.weights.best.hdf5
aucroc =  0.9712828146698964
5/5 [==============================] - 3s 563ms/step - loss: 0.4807 - acc: 0.9121 - val_loss: 0.0923 - val_acc: 0.9228
Epoch 1260/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4240 - acc: 0.9185
Epoch 01260: val_acc did not improve from 0.92284
aucroc =  0.9695234343357124
5/5 [==============================] - 3s 530ms/step - loss: 0.4199 - acc: 0.9190 - val_loss: 0.0960 - val_acc: 0.9214
Epoch 1261/1000000


3/5 [=================>............] - ETA: 0s - loss: 0.5010 - acc: 0.9048
Epoch 01285: val_acc did not improve from 0.92343
aucroc =  0.9696472086587309
5/5 [==============================] - 3s 539ms/step - loss: 0.4389 - acc: 0.9209 - val_loss: 0.0929 - val_acc: 0.9225
Epoch 1286/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5680 - acc: 0.8996
Epoch 01286: val_acc did not improve from 0.92343
aucroc =  0.9651580717764691
5/5 [==============================] - 3s 534ms/step - loss: 0.5321 - acc: 0.9088 - val_loss: 0.0994 - val_acc: 0.9219
Epoch 1287/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5684 - acc: 0.8994
Epoch 01287: val_acc did not improve from 0.92343
aucroc =  0.9726259848229963
5/5 [==============================] - 3s 537ms/step - loss: 0.4929 - acc: 0.9128 - val_loss: 0.0902 - val_acc: 0.9233
Epoch 1288/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5587 - acc: 0.8960
Epoch 01288: val_acc did not improve from 0.9

Epoch 1313/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5496 - acc: 0.8925
Epoch 01313: val_acc did not improve from 0.92350
aucroc =  0.970374247911572
5/5 [==============================] - 3s 538ms/step - loss: 0.5410 - acc: 0.8938 - val_loss: 0.0960 - val_acc: 0.9220
Epoch 1314/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4540 - acc: 0.9130
Epoch 01314: val_acc did not improve from 0.92350
aucroc =  0.9703503332525133
5/5 [==============================] - 3s 537ms/step - loss: 0.4891 - acc: 0.9045 - val_loss: 0.0941 - val_acc: 0.9225
Epoch 1315/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5042 - acc: 0.9033
Epoch 01315: val_acc did not improve from 0.92350
aucroc =  0.9708818548807774
5/5 [==============================] - 3s 538ms/step - loss: 0.5631 - acc: 0.8937 - val_loss: 0.0921 - val_acc: 0.9230
Epoch 1316/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5019 - acc: 0.9034
Epoch 01316: val_acc did no

Epoch 1341/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4891 - acc: 0.9102
Epoch 01341: val_acc did not improve from 0.92356
aucroc =  0.9639336245011129
5/5 [==============================] - 3s 538ms/step - loss: 0.4887 - acc: 0.9089 - val_loss: 0.1019 - val_acc: 0.9213
Epoch 1342/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4942 - acc: 0.9216
Epoch 01342: val_acc did not improve from 0.92356
aucroc =  0.9644695889552796
5/5 [==============================] - 3s 542ms/step - loss: 0.4648 - acc: 0.9203 - val_loss: 0.1052 - val_acc: 0.9201
Epoch 1343/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5536 - acc: 0.8978
Epoch 01343: val_acc did not improve from 0.92356
aucroc =  0.9706498145141248
5/5 [==============================] - 3s 554ms/step - loss: 0.5072 - acc: 0.9049 - val_loss: 0.0949 - val_acc: 0.9223
Epoch 1344/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4608 - acc: 0.9113
Epoch 01344: val_acc did n

Epoch 1369/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.6824 - acc: 0.8744
Epoch 01369: val_acc did not improve from 0.92359
aucroc =  0.9708685668190098
5/5 [==============================] - 3s 540ms/step - loss: 0.6525 - acc: 0.8790 - val_loss: 0.0931 - val_acc: 0.9228
Epoch 1370/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5738 - acc: 0.8889
Epoch 01370: val_acc did not improve from 0.92359
aucroc =  0.966758888278684
5/5 [==============================] - 3s 538ms/step - loss: 0.5739 - acc: 0.8900 - val_loss: 0.1017 - val_acc: 0.9206
Epoch 1371/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5225 - acc: 0.9061
Epoch 01371: val_acc did not improve from 0.92359
aucroc =  0.9694546370389073
5/5 [==============================] - 3s 538ms/step - loss: 0.5397 - acc: 0.9008 - val_loss: 0.0953 - val_acc: 0.9220
Epoch 1372/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5245 - acc: 0.9025
Epoch 01372: val_acc did no

...

Epoch 7669/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4223 - acc: 0.9188
Epoch 07669: val_acc did not improve from 0.92428
aucroc =  0.9741188927076121
5/5 [==============================] - 3s 539ms/step - loss: 0.4268 - acc: 0.9164 - val_loss: 0.0999 - val_acc: 0.9203
Epoch 7670/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5303 - acc: 0.8928
Epoch 07670: val_acc did not improve from 0.92428
aucroc =  0.9743877424698248
5/5 [==============================] - 3s 539ms/step - loss: 0.4989 - acc: 0.9003 - val_loss: 0.0990 - val_acc: 0.9204
Epoch 7671/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4105 - acc: 0.9146
Epoch 07671: val_acc did not improve from 0.92428
aucroc =  0.9760485024230331
5/5 [==============================] - 3s 538ms/step - loss: 0.4069 - acc: 0.9111 - val_loss: 0.0936 - val_acc: 0.9213
Epoch 7672/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3781 - acc: 0.9249
Epoch 07672: val_acc did n

Epoch 7697/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5110 - acc: 0.9016
Epoch 07697: val_acc did not improve from 0.92428
aucroc =  0.975675277039647
5/5 [==============================] - 3s 570ms/step - loss: 0.4609 - acc: 0.9102 - val_loss: 0.0882 - val_acc: 0.9235
Epoch 7698/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.2698 - acc: 0.9454
Epoch 07698: val_acc did not improve from 0.92428
aucroc =  0.9750710182240275
5/5 [==============================] - 3s 556ms/step - loss: 0.3652 - acc: 0.9235 - val_loss: 0.0884 - val_acc: 0.9235
Epoch 7699/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3987 - acc: 0.9236
Epoch 07699: val_acc did not improve from 0.92428
aucroc =  0.9761308540628995
5/5 [==============================] - 3s 559ms/step - loss: 0.4997 - acc: 0.9027 - val_loss: 0.0873 - val_acc: 0.9235
Epoch 7700/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.7238 - acc: 0.8586
Epoch 07700: val_acc did no

Epoch 7725/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4494 - acc: 0.9100
Epoch 07725: val_acc did not improve from 0.92428
aucroc =  0.9767892294115507
5/5 [==============================] - 3s 557ms/step - loss: 0.4380 - acc: 0.9102 - val_loss: 0.0923 - val_acc: 0.9217
Epoch 7726/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5094 - acc: 0.8921
Epoch 07726: val_acc did not improve from 0.92428
aucroc =  0.9771446292859448
5/5 [==============================] - 3s 554ms/step - loss: 0.4461 - acc: 0.9058 - val_loss: 0.0906 - val_acc: 0.9222
Epoch 7727/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.5055 - acc: 0.8969
Epoch 07727: val_acc did not improve from 0.92428
aucroc =  0.9773341133524466
5/5 [==============================] - 3s 549ms/step - loss: 0.4591 - acc: 0.9041 - val_loss: 0.0874 - val_acc: 0.9235
Epoch 7728/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4023 - acc: 0.9137
Epoch 07728: val_acc did n

Epoch 7753/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3970 - acc: 0.9143
Epoch 07753: val_acc did not improve from 0.92428
aucroc =  0.9763602556561436
5/5 [==============================] - 3s 553ms/step - loss: 0.3994 - acc: 0.9188 - val_loss: 0.0908 - val_acc: 0.9224
Epoch 7754/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4629 - acc: 0.9023
Epoch 07754: val_acc did not improve from 0.92428
aucroc =  0.9770341988001554
5/5 [==============================] - 3s 545ms/step - loss: 0.4351 - acc: 0.9091 - val_loss: 0.0927 - val_acc: 0.9219
Epoch 7755/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3880 - acc: 0.9208
Epoch 07755: val_acc did not improve from 0.92428
aucroc =  0.9773941651122371
5/5 [==============================] - 3s 550ms/step - loss: 0.4082 - acc: 0.9138 - val_loss: 0.0911 - val_acc: 0.9222
Epoch 7756/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3824 - acc: 0.9231
Epoch 07756: val_acc did n

Epoch 7781/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3638 - acc: 0.9272
Epoch 07781: val_acc did not improve from 0.92428
aucroc =  0.9762489033760853
5/5 [==============================] - 3s 543ms/step - loss: 0.3837 - acc: 0.9225 - val_loss: 0.1021 - val_acc: 0.9189
Epoch 7782/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.4368 - acc: 0.9092
Epoch 07782: val_acc did not improve from 0.92428
aucroc =  0.9766932505640216
5/5 [==============================] - 3s 548ms/step - loss: 0.4176 - acc: 0.9104 - val_loss: 0.0912 - val_acc: 0.9223
Epoch 7783/1000000
3/5 [=================>............] - ETA: 0s - loss: 0.3139 - acc: 0.9320
Epoch 07783: val_acc did not improve from 0.92428
aucroc =  0.976664886501165
5/5 [==============================] - 3s 543ms/step - loss: 0.3569 - acc: 0.9272 - val_loss: 0.0903 - val_acc: 0.9226
Epoch 7784/1000000
4/5 [=======================>......] - ETA: 0s - loss: 0.5517 - acc: 0.8899
Epoch 07784: val_acc did no